### OpenSearch 

#### Importing all required libraries

In [21]:
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_community.document_loaders.text import TextLoader
from langchain_community.vectorstores import OpenSearchVectorSearch
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from opensearchpy import OpenSearch, RequestsHttpConnection
import requests

#### Loading all environment variables

In [15]:
from dotenv import load_dotenv

load_dotenv()

True

#### Creating Connection to OpenSearch (Amazon Open Search Service) 

In [16]:
from opensearchpy import OpenSearch, RequestsHttpConnection

host = 'localhost'
port = 9200
auth = ('admin', 'opensearch123@KB')

# creating the client with SSL/TLS enabled
client = OpenSearch(
    hosts=[{"host": host, "port": port}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=False,
    connection_class=RequestsHttpConnection,
    timeout=300
)

c:\Users\karthik.kolluri\AppData\Local\anaconda3\envs\llm_apps\lib\site-packages\opensearchpy\connection\http_requests.py:160: UserWarning: Connecting to https://localhost:9200 using SSL with verify_certs=False is insecure.
  warnings.warn(


In [17]:
client.info()

{'name': '41db466f3b84',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': '_UVT0oRqQwOQV-PCFjgA7w',
 'version': {'distribution': 'opensearch',
  'number': '2.15.0',
  'build_type': 'tar',
  'build_hash': '61dbcd0795c9bfe9b81e5762175414bc38bbcadf',
  'build_date': '2024-06-20T03:26:49.193630411Z',
  'build_snapshot': False,
  'lucene_version': '9.10.0',
  'minimum_wire_compatibility_version': '7.10.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'The OpenSearch Project: https://opensearch.org/'}

#### Creating Index

In [18]:
index_name = "semantic-index"

if client.indices.exists(index_name):
    client.indices.delete(index_name)
    print(f"Index {index_name} deleted!")

Index semantic-index deleted!


In [19]:
index_body = {
                'settings': {
                    "index.knn": True
                },
                "mappings": {
                    "properties": {
                    "vector_field": {
                        "type": "knn_vector",
                        "dimension": 1536,
                        "method": {
                        "engine": "nmslib",
                        "name": "hnsw",
                        "space_type": "innerproduct"
                        }
                    }
                    }
                }
                }


response = client.indices.create(index=index_name, 
                                body=index_body)

### RAG Pipeline 

#### 1. Data Ingestion 

In [22]:
# document loader 
loader = TextLoader(
    file_path="sample_doc/docs.txt"
)

document = loader.load()

document

[Document(page_content='Italy, a European country with a long Mediterranean coastline, has left a powerful mark on Western culture and cuisine. \nIts capital, Rome, is home to the Vatican as well as landmark art and ancient ruins. \nOther major cities include Florence, with Renaissance masterpieces such as Michelangelo’s "David" and Brunelleschi\'s Duomo; Venice, \nthe city of canals; and Milan, \nItaly’s fashion capital.\nourists and travel experts have long agreed that Italy is a special place, so much so that the country has become a de facto bucket list destination for just about everyone. Famous for its incredible food, rich historical sites, highly regarded art, charming small towns and picturesque cities, countrysides and coastlines, it\'s safe to say Italy\'s offerings are unmatched. U.S. News rounded up the best places to visit in Italy considering a number of factors, from affordability and seasonality to sights and crowds, to help you decide exactly where to go. Have a favor

In [27]:
document[0]

Document(page_content='Italy, a European country with a long Mediterranean coastline, has left a powerful mark on Western culture and cuisine. \nIts capital, Rome, is home to the Vatican as well as landmark art and ancient ruins. \nOther major cities include Florence, with Renaissance masterpieces such as Michelangelo’s "David" and Brunelleschi\'s Duomo; Venice, \nthe city of canals; and Milan, \nItaly’s fashion capital.\nourists and travel experts have long agreed that Italy is a special place, so much so that the country has become a de facto bucket list destination for just about everyone. Famous for its incredible food, rich historical sites, highly regarded art, charming small towns and picturesque cities, countrysides and coastlines, it\'s safe to say Italy\'s offerings are unmatched. U.S. News rounded up the best places to visit in Italy considering a number of factors, from affordability and seasonality to sights and crowds, to help you decide exactly where to go. Have a favori

In [23]:
len(document[0].page_content)

4832

In [24]:
document[0].page_content

'Italy, a European country with a long Mediterranean coastline, has left a powerful mark on Western culture and cuisine. \nIts capital, Rome, is home to the Vatican as well as landmark art and ancient ruins. \nOther major cities include Florence, with Renaissance masterpieces such as Michelangelo’s "David" and Brunelleschi\'s Duomo; Venice, \nthe city of canals; and Milan, \nItaly’s fashion capital.\nourists and travel experts have long agreed that Italy is a special place, so much so that the country has become a de facto bucket list destination for just about everyone. Famous for its incredible food, rich historical sites, highly regarded art, charming small towns and picturesque cities, countrysides and coastlines, it\'s safe to say Italy\'s offerings are unmatched. U.S. News rounded up the best places to visit in Italy considering a number of factors, from affordability and seasonality to sights and crowds, to help you decide exactly where to go. Have a favorite destination in Ital

In [25]:
document[0].metadata

{'source': 'sample_doc/docs.txt'}

### 2. Splitting document into chunks

In [10]:
text_splitter = (
    chunk_size=1000,
    chunk_overlap=100
)

chunks = text_splitter.split_documents(document)

len(chunks)

15

#### 3. Embedding model 

In [7]:
embedding_model = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)

#### 4. Indexing Documents into Vector store

In [8]:
index_name = "semantic-index"

vector_store = OpenSearchVectorSearch(
    index_name= index_name,
    embedding_function=embedding_model,
    opensearch_url="https://localhost:9200",
    http_auth=("admin", "opensearch123@KB"),
    use_ssl = False,
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False,
)

In [11]:
doc_ids = ["doc"+str(i) for i in range(len(chunks))]

print(doc_ids)

['doc0', 'doc1', 'doc2', 'doc3', 'doc4', 'doc5', 'doc6', 'doc7', 'doc8', 'doc9', 'doc10', 'doc11', 'doc12', 'doc13', 'doc14']


In [12]:
vector_store.add_documents(
    documents=chunks,
    vector_field="vector_field",
    ids=doc_ids
)

['doc0',
 'doc1',
 'doc2',
 'doc3',
 'doc4',
 'doc5',
 'doc6',
 'doc7',
 'doc8',
 'doc9',
 'doc10',
 'doc11',
 'doc12',
 'doc13',
 'doc14']

#### 5. Semantic Search

In [21]:
docs = vector_store.similarity_search("what is Scaled Dot-Product?", 
                                      k=3,
                                      vector_field="vector_field",
                                      text_field="text",
                                      metadata_field="metadata")

docs

[Document(page_content='Scaled Dot-Product Attention\n Multi-Head Attention\nFigure 2: (left) Scaled Dot-Product Attention. (right) Multi-Head Attention consists of several\nattention layers running in parallel.\nof the values, where the weight assigned to each value is computed by a compatibility function of the\nquery with the corresponding key.\n3.2.1 Scaled Dot-Product Attention\nWe call our particular attention "Scaled Dot-Product Attention" (Figure 2). The input consists of\nqueries and keys of dimension dk, and values of dimension dv. We compute the dot products of the\nquery with all keys, divide each by√dk, and apply a softmax function to obtain the weights on the\nvalues.\nIn practice, we compute the attention function on a set of queries simultaneously, packed together\ninto a matrix Q. The keys and values are also packed together into matrices KandV. We compute\nthe matrix of outputs as:\nAttention( Q, K, V ) = softmax(QKT\n√dk)V (1)\nThe two most commonly used attention fu

In [24]:
def pretty_print(docs):
    for i,doc in enumerate(docs):
        print(f"Document: {i}")
        print("\n")
        print("Content: \n")
        print(doc.page_content)
        print("\n")
        print("Metadata: \n")
        print(doc.metadata)

In [25]:
pretty_print(docs)

Document: 0


Content: 

Scaled Dot-Product Attention
 Multi-Head Attention
Figure 2: (left) Scaled Dot-Product Attention. (right) Multi-Head Attention consists of several
attention layers running in parallel.
of the values, where the weight assigned to each value is computed by a compatibility function of the
query with the corresponding key.
3.2.1 Scaled Dot-Product Attention
We call our particular attention "Scaled Dot-Product Attention" (Figure 2). The input consists of
queries and keys of dimension dk, and values of dimension dv. We compute the dot products of the
query with all keys, divide each by√dk, and apply a softmax function to obtain the weights on the
values.
In practice, we compute the attention function on a set of queries simultaneously, packed together
into a matrix Q. The keys and values are also packed together into matrices KandV. We compute
the matrix of outputs as:
Attention( Q, K, V ) = softmax(QKT
√dk)V (1)
The two most commonly used attention functions are add

#### 6. Hybrid Search

Create a search pipeline in OpenSearch to normalize the search results from the text and vector queries. The search pipelines combines the results from each subquery.

Create Search-Pipeline

In [19]:
path="_search/pipeline/nlp-search-pipeline"
host = "https://localhost:9200/"
auth = ('admin', 'opensearch123@KB')
url = host + path
payload = {
  "description": "Post processor for hybrid search",
  "phase_results_processors": [
    {
      "normalization-processor": {
        "normalization": {
          "technique": "min_max"
        },
        "combination": {
          "technique": "arithmetic_mean",
          "parameters": {
            "weights": [
              0.3,
              0.7
            ]
          }
        }
      }
    }
  ]
}
r = requests.put(url, auth=("admin", "opensearch123@KB"), json=payload, verify=False)
print(r.status_code)
print(r.text)

c:\Users\karthik.kolluri\AppData\Local\anaconda3\envs\llm_apps\lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


200
{"acknowledged":true}


In [6]:
def embed_query(query):
    embedding_model = OpenAIEmbeddings(
        model="text-embedding-ada-002"
    )
    embedding = embedding_model.embed_query(query)
    return embedding

In [7]:
query = "what is Scaled Dot-Product Attention ?"

embeded_query = embed_query(query)

In [15]:
embeded_query

[-0.034961410785981736,
 0.021099869716581798,
 0.02922510618811345,
 -0.00889055547395583,
 -0.015420785579202274,
 0.031671260384910783,
 -0.02134305481101383,
 -0.022988128148904108,
 -0.034560871287850806,
 -0.030269372141452536,
 0.04763563847611108,
 0.019869639129299437,
 -0.021071259486337417,
 0.018668020634906655,
 0.01338947599565806,
 0.004709920038431935,
 0.007789070457111883,
 0.012416736549252523,
 -0.029182192705392077,
 -0.0244615438306185,
 -0.032815662144105195,
 0.011215117123537144,
 0.010614306945018154,
 0.003958907548113848,
 0.0013679152248575912,
 0.00730985282580891,
 0.03416033178971988,
 -0.030898791618893306,
 -0.016021594826398665,
 -0.01659379570599587,
 0.018467749023195996,
 -1.1734566977988571e-06,
 0.0032472341243817834,
 -0.010707289261989791,
 -0.007903509981105505,
 0.012495413751101971,
 0.00320789529062641,
 -0.02075654881629443,
 0.014054658260904306,
 -0.008089475546371377,
 0.010442647426197072,
 0.00581498086839513,
 0.002186876313514132,
 

In [10]:
path = "semantic-index/_search?search_pipeline=nlp-search-pipeline" 
host = "https://localhost:9200/"
auth = ('admin', 'opensearch123@KB')
url = host + path
query_ = "what is the technique ?"

payload = {
  "_source": {
    "exclude": [
      "vector_field"
    ]
  },
  "query": {
    "hybrid": {
      "queries": [
        {
          "match": {
            "caption": {
              "query": query_
            }
          }
        },
        {
          "knn": {
            "vector_field": {
            "vector": embeded_query,
            "k": 4
            }
          }
        }
      ]
    }
  },"size":4
}

r = requests.get(url, auth=auth, json=payload, verify=False)
print(r.status_code)
print(r.text)

result = r.json()

200
{"took":31,"timed_out":false,"_shards":{"total":1,"successful":1,"skipped":0,"failed":0},"hits":{"total":{"value":4,"relation":"eq"},"max_score":0.7,"hits":[{"_index":"semantic-index","_id":"doc3","_score":0.7,"_source":{"metadata":{"source":"sample_doc/doc.pdf","page":3},"text":"Scaled Dot-Product Attention\n Multi-Head Attention\nFigure 2: (left) Scaled Dot-Product Attention. (right) Multi-Head Attention consists of several\nattention layers running in parallel.\nof the values, where the weight assigned to each value is computed by a compatibility function of the\nquery with the corresponding key.\n3.2.1 Scaled Dot-Product Attention\nWe call our particular attention \"Scaled Dot-Product Attention\" (Figure 2). The input consists of\nqueries and keys of dimension dk, and values of dimension dv. We compute the dot products of the\nquery with all keys, divide each by√dk, and apply a softmax function to obtain the weights on the\nvalues.\nIn practice, we compute the attention functio

c:\Users\karthik.kolluri\AppData\Local\anaconda3\envs\llm_apps\lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [11]:
result['hits']["hits"]

[{'_index': 'semantic-index',
  '_id': 'doc3',
  '_score': 0.7,
  '_source': {'metadata': {'source': 'sample_doc/doc.pdf', 'page': 3},
   'text': 'Scaled Dot-Product Attention\n Multi-Head Attention\nFigure 2: (left) Scaled Dot-Product Attention. (right) Multi-Head Attention consists of several\nattention layers running in parallel.\nof the values, where the weight assigned to each value is computed by a compatibility function of the\nquery with the corresponding key.\n3.2.1 Scaled Dot-Product Attention\nWe call our particular attention "Scaled Dot-Product Attention" (Figure 2). The input consists of\nqueries and keys of dimension dk, and values of dimension dv. We compute the dot products of the\nquery with all keys, divide each by√dk, and apply a softmax function to obtain the weights on the\nvalues.\nIn practice, we compute the attention function on a set of queries simultaneously, packed together\ninto a matrix Q. The keys and values are also packed together into matrices KandV. We

In [22]:
url = "https://localhost:9200/"
path = "semantic-index/_search"
url = host + path

auth = ('admin', 'opensearch123@KB')


payload = {
     "query": 
       {"match_all": {}}
  
}

r = requests.get(url, auth=auth, json=payload, verify=False)
print(r.status_code)
print(r.text)

result = r.json()

200
{"took":3,"timed_out":false,"_shards":{"total":1,"successful":1,"skipped":0,"failed":0},"hits":{"total":{"value":15,"relation":"eq"},"max_score":1.0,"hits":[{"_index":"semantic-index","_id":"doc0","_score":1.0,"_source":{"vector_field":[-0.03484686226964574,0.011483308374112735,0.03935940716637783,-0.01642064441479642,0.025557152057482067,0.012332892524661162,-0.009930381005978112,-0.012200580452666187,-0.009220073357908289,-0.04039004974569356,0.029693649373067052,0.03033431923555888,0.010048765148328444,0.03306412825332625,-0.009470769572253596,0.006222156583925238,0.03353766854801802,-0.0006054154599669506,-0.008913665424984406,-0.012256290308599538,-0.024721494905255676,0.0025191552519044326,-0.013885819892795784,0.014345431326520301,-0.004717975572562951,0.022019539884132373,0.02665743149105319,-0.022855197036358764,-0.026323168630162635,-0.03055716052193751,0.025292526050846895,0.018788336575029166,-0.011337068372473115,-0.01803624513802541,-0.01598888884036119,-0.01261144459

c:\Users\karthik.kolluri\AppData\Local\anaconda3\envs\llm_apps\lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
